In [8]:
import os
import io
import spacy
import pathlib 
import warnings
import fr_core_news_sm

import pandas as pd 
import tensorflow as tf 
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from spacy.lang.fr.stop_words import STOP_WORDS
warnings.filterwarnings('ignore')
nlp = fr_core_news_sm.load()

In [4]:
df = pd.read_csv("https://go.aws/314bBDq")

In [5]:
df

,user_id,review,stars,date_format,time_of_day,hour_of_day,day_of_week,review_format,review_lang,month_year,review_len,review_nb_words
0,efb62a167fee5cf3678b24427de8e31f,"Génial, fabuleux, exceptionnel ! J'aimerais qu...",5,2017-09-29 18:17:00,18:17,18,Ven,génial fabuleux exceptionnel j aimerais qu...,french,2017-09,115,19
1,e3be4f9c9e0b9572bfb2a5f88497bb14,NaN,2,2017-09-29 17:29:00,17:29,17,Ven,NaN,NaN,2017-09,0,0
2,1b8e5760162d867e9b9ca80f645bdc60,"Toujours aussi magic, féerique !",5,2017-09-29 16:46:00,16:46,16,Ven,toujours aussi magic féerique,french,2017-09,32,4
3,fa330e5891a1bb486c3e9bf95c098726,NaN,5,2017-09-29 15:52:00,15:52,15,Ven,NaN,NaN,2017-09,0,0
4,c1a693206aee1a2412d4bd9e45b80ec5,NaN,3,2017-09-29 15:29:00,15:29,15,Ven,NaN,NaN,2017-09,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
299630,299be03d0583edfb9625a7947fbc631a,NaN,5,2012-11-11 11:46:00,11:46,11,Dim,NaN,NaN,2012-11,0,0
299631,39b4e66e3b78d4f8ce60a6b4801b862d,NaN,5,2012-11-11 11:46:00,11:46,11,Dim,NaN,NaN,2012-11,0,0
299632,924eb5ec58470cd00c16060e6ee3c316,NaN,5,2012-11-11 11:46:00,11:46,11,Dim,NaN,NaN,2012-11,0,0
299633,5b484e48319355c12a941577d74a5839,NaN,5,2012-11-11 11:45:00,11:45,11,Dim,NaN,NaN,2012-11,0,0


In [6]:
df = df[(df["review_lang"] == "french")]

In [7]:
df = df[["review", "stars"]]

In [9]:
df["review_clean"] = df["review"].apply(lambda x:''.join(ch for ch in x if ch.isalnum() or ch==" " or ch=="'"))
df["review_clean"] = df["review_clean"].apply(lambda x: x.replace(" +"," ").lower().strip())
df["review_clean"] = df["review_clean"].apply(lambda x: " ".join([token.lemma_ for token in nlp(x) if (token.lemma_ not in STOP_WORDS) and (token.text not in STOP_WORDS)]))

In [10]:
mask = df.review_clean.apply(lambda x: type(x)==str)
mask.value_counts()

True    8474
Name: review_clean, dtype: int64

In [11]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=1000, oov_token="out_of_vocab") 
tokenizer.fit_on_texts(df.review_clean)
df["review_encoded"] = tokenizer.texts_to_sequences(df.review_clean)

In [12]:
reviews_pad = tf.keras.preprocessing.sequence.pad_sequences(df.review_encoded, padding="post")

In [13]:
xtrain, xval, ytrain, yval = train_test_split(reviews_pad,df.stars, test_size=0.3)

In [14]:
train = tf.data.Dataset.from_tensor_slices((xtrain, ytrain))
val = tf.data.Dataset.from_tensor_slices((xval, yval))

In [15]:
train_batch = train.shuffle(len(train)).batch(64)
val_batch = val.shuffle(len(val)).batch(64)

In [16]:
for review, star in train_batch.take(1):
  print(review, star)

tf.Tensor(
[[ 10  22  51 ...   0   0   0]
 [ 26 803  73 ...   0   0   0]
 [464  43   6 ...   0   0   0]
 ...
 [358  10 591 ...   0   0   0]
 [ 14  62  14 ...   0   0   0]
 [ 20  54  64 ...   0   0   0]], shape=(64, 443), dtype=int32) tf.Tensor(
[5 5 1 1 5 4 5 5 4 5 5 2 3 5 4 5 5 3 3 5 4 4 4 4 4 3 2 5 4 3 4 3 2 5 5 1 5
 5 5 1 3 4 5 5 4 5 3 3 5 5 5 3 5 5 5 3 5 5 5 2 3 5 2 5], shape=(64,), dtype=int64)


In [ ]:
embedding = 64
vocab_size = tokenizer.num_words

